In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [3]:
sys.path.insert(0, os.path.abspath('../palin/python'))

In [4]:
#from palin.simulation import Experiment
from palin.simulation.experiment import Experiment
from palin.simulation.analyser import Analyser
from palin.simulation.experiments.simple_experiment import SimpleExperiment
from palin.simulation.experiments.double_pass_experiment import DoublePassExperiment
from palin.simulation.trial import Int2Trial, Int1Trial 
from palin.simulation.observers.linear_observer import LinearObserver
from palin.simulation.analysers.kernel_distance import KernelDistance
from palin.simulation.analysers.internal_noise_value import InternalNoiseValue
from palin.kernels.classification_images import ClassificationImage
from palin.internal_noise.double_pass import DoublePass
from palin.internal_noise.intercept_method import InterceptMethod
from palin.simulation.simulation import Simulation as Sim
from palin.simulation.analysers.agreement_statistics import AgreementStatistics
from palin.internal_noise.agreement_method import AgreementMethod
from palin.kernels.glm_kernel import GLMKernel
from palin.internal_noise.glm_method import GLMMethod


# Three new algorithms to estimate observer internal noise without double-pass measurements

&copy; Aynaz Adl Zarrabi, Jean-Julien Aucouturier, Ladislas Nalborczyk, Marie Villain

## Methods

### Consistency methods

#### Forward and inverse models

- The ideal observer model provides a generative model by which one can simulate the responses given by an observer with a given kernel and internal noise level, to arbitrary stimuli.  
- From these responses, one can evaluate a probability of agreement between duplicated trials (forward model). This model is stochastic: for a given observer, and a given experiment, responses have stochasticity because of internal noise realizations, and so does probability of agreement 
- the task of internal noise estimation from a real observer's response is the inverse model: from an observation of probability of agreement, find the most plausible internal noise level (hidden variable) that would forward-generate this probability
- to extend to the case where observers are biased (i.e a non null response criteria), one can also observe probability of answering the first interval (in a 2AFC design). In which case the forward model maps two hidden variables (internal noise and criteria) to two observations (probability of agreement, and probability of first interval)
- to deal with stochasticity, we compute the expected values of the observations (prob_agree, prob_int1) over several realizations of the same simulated experiment (forward model), and therefore aim to estimate the internal noise and criteria for which the observed probabilities are closest to their expected values
- a typical practical implementation of this procedure is to simulate observers with an arbitrary kernel (ex. a simple scalar) and a grid of internal noise and criteria values, encountering a single experiment for several runs (ex. 100 or 1000), compute the average corresponding probabilities for each internal noise and criteria; store this as a lookup table; then given a real observation of probabilities for a real observer, find the pair of internal noise and criteria that correspond to the closest pair of probabilities in the table. 

In [ ]:
AgreementMethod.build_model(agreement_model_file = './double_pass_model.csv',
                            internal_noise_range=np.arange(0,5,.1),
                            criteria_range=np.arange(-5,5,.5), 
                            n_repeated_trials=100, 
                            n_runs=100) 

In [20]:
model_df = pd.read_csv('./agreement_model_large.csv')
model_df

,internal_noise_std,criteria,prob_agree,prob_first
0,0.0,-5,1.00000,0.99988
1,0.0,-4,1.00000,0.99760
2,0.0,-3,1.00000,0.98288
3,0.0,-2,1.00000,0.92036
4,0.0,-1,1.00000,0.75676
...,...,...,...,...
495,4.9,0,0.52124,0.49998
496,4.9,1,0.53232,0.42424
497,4.9,2,0.56752,0.34608
498,4.9,3,0.61634,0.27841
